In [36]:
import sys
from pathlib import Path
from dataclasses import dataclass, field
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from utils import load_config, load_datasets
import pandas as pd
import numpy as np
import re
import sklearn as skt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor, XGBClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from noise_remover import NoiseRemover
from titanic_preprocessor import TitanicPreprocessor
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from xgboost.callback import EarlyStopping

load_config()
dataFrameTrain,dataFrameTest = load_datasets()

X=dataFrameTrain.drop(columns=["Survived"])
Y=dataFrameTrain["Survived"]

#print(dataFrameTest)


ids_to_predict=dataFrameTest["PassengerId"]


seeds = [1, 42, 99, 145, 3255, 5000]
for seed in seeds:
    model= XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",       # modela conteos
        tree_method="hist",  
        colsample_bytree = 0.84,
        n_estimators=31,
        learning_rate=0.2,
        subsample=1.0,
        max_depth=2,
        reg_lambda=10,            # L2 ayuda a estabilizar
        reg_alpha=15,
        enable_categorical=True,
        seed=seed
    )
  


    data_preprocessor = TitanicPreprocessor()


    pipeline = Pipeline([('pre-processor', data_preprocessor), ('nosie remover', NoiseRemover()),('rfb', model)])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, Y, scoring="accuracy", cv=cv, n_jobs=-1)
    print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f},, {scores.mean()+scores.std():.4f}")


CV Accuracy: 0.8238 ± 0.0102,, 0.8341
CV Accuracy: 0.8204 ± 0.0099,, 0.8304
CV Accuracy: 0.8216 ± 0.0095,, 0.8311
CV Accuracy: 0.8238 ± 0.0102,, 0.8341
CV Accuracy: 0.8238 ± 0.0102,, 0.8341
CV Accuracy: 0.8227 ± 0.0096,, 0.8323


In [45]:

import sys
from pathlib import Path
from dataclasses import dataclass, field
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from utils import load_config, load_datasets
import pandas as pd
import numpy as np
import re
import sklearn as skt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor, XGBClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from noise_remover import NoiseRemover
from titanic_preprocessor import TitanicPreprocessor
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from xgboost.callback import EarlyStopping

load_config()
dataFrameTrain,dataFrameTest = load_datasets()

X=dataFrameTrain.drop(columns=["Survived"])
Y=dataFrameTrain["Survived"]

#print(dataFrameTest)


ids_to_predict=dataFrameTest["PassengerId"]


model= XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",       # modela conteos
    tree_method="hist",  
    colsample_bytree = 0.8,
    n_estimators=14,
    learning_rate=0.1,
    subsample=1.0,
    max_depth=2,
    reg_lambda=10,            # L2 ayuda a estabilizar
    reg_alpha=15,
    enable_categorical=True,
    seed=seed
)

#'max_depth': 2, 'learning_rate': 0.009, 'subsample': 0.8, 'colsample_bytree': 1.0 'n_estimators': 5000 , result: 0.02004158700209857, cv: 0.8438350774622557, ov_cof: 0.8237934904601572
#Best score: 0.02004158700209857
#Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.009, 'max_depth': 2, 'reg_alpha': 15, 'reg_lambda': 0, 'subsample': 0.8}

data_preprocessor = TitanicPreprocessor()


pipeline = Pipeline([('pre-processor', data_preprocessor), ('nosie remover', NoiseRemover()),('rfb', model)])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, Y, scoring="accuracy", cv=cv, n_jobs=-1)
print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f},, {scores.mean()+scores.std():.4f}")


CV Accuracy: 0.8103 ± 0.0239,, 0.8342


In [43]:
pipeline.fit(X,Y)
predictions_array=pipeline.predict(dataFrameTest)
results = pd.DataFrame({'PassengerId' : ids_to_predict, 'Survived' : pd.Series(predictions_array)})
results.to_csv("ignadormido10.csv", index=False)
